# DocumentSummaryIndex

## 准备

In [1]:
%%time

from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

CPU times: user 2.62 s, sys: 352 ms, total: 2.97 s
Wall time: 2.61 s


In [2]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 122 ms, sys: 8.11 ms, total: 130 ms
Wall time: 129 ms


In [3]:
%%time

# embedding
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    # model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    model_name="chatfire/bge-m3:q8_0",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 532 ms, sys: 20 ms, total: 552 ms
Wall time: 552 ms


In [4]:
%%time

data_titles = ["故宫","颐和园"]

data_docs = []
for data_title in data_titles:
    docs = SimpleDirectoryReader(
        input_files=[f"data/{data_title}.txt"]
    ).load_data()
    docs[0].doc_id = data_title
    data_docs.extend(docs)

CPU times: user 6.1 ms, sys: 4.26 ms, total: 10.4 ms
Wall time: 10.5 ms


In [5]:
data_docs[0]

Document(id_='故宫', embedding=None, metadata={'file_path': 'data/故宫.txt', 'file_name': '故宫.txt', 'file_type': 'text/plain', 'file_size': 1571, 'creation_date': '2024-06-17', 'last_modified_date': '2024-06-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='故宫\n\n故宫，又称紫禁城，是中国明清两朝二十四位皇帝的皇宫。故宫始建于明成祖永乐四年（1406年），永乐十八年（1420年）落成。位于北京中轴线中心的故宫，占地面积72万平方米，建筑面积约15万平方米，有大小宫殿七十多座，房屋九千余间。是世界上现存规模最大、最完整的宫殿型建筑。北京故宫是第一批全国重点文物保护单位、第一批国家5A级旅游景区，1987年被选入《世界文化遗产》名录。故宫现为故宫博物院，藏品主要以明、清两代宫廷收藏为基础，是国家一级博物馆。\n\n故宫南北长961米，东西宽753米，四面围有高10米的城墙，外有59米宽的护城河环绕。有四座城门，南面为午门，北面为神武门，东面为东华门，西面为西华门。城墙的四角，各有一座风姿绰约的角楼，民间有九梁十八柱七十二条脊之说，来形容其结构的复杂。城内的建筑分为外朝和内廷两部分。外朝的中心为太和殿、中和殿、保和殿，统称三大殿，是国家举行大典礼的地方。三大殿左右两翼辅以文华殿、武英殿两组建筑。内廷的中心是乾清宫、交泰殿、坤宁宫，统称后三宫，是皇帝和皇后居住的正宫。其后为御花园。后三宫两侧排列着东、西六宫，是后妃们居住休息的地方。东六宫东侧是天穹宝殿等

In [6]:
%%time

splitter = SentenceSplitter(chunk_size=1024)

CPU times: user 235 ms, sys: 19.9 ms, total: 255 ms
Wall time: 255 ms


## 创建文档摘要索引

### 使用默认提示词将生成英文内容

In [7]:
%%time

# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
    # text_qa_template=qa_prompt,
    # refine_template=refine_prompt,
    # summary_template=summary_prompt,
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    data_docs,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/2 [00:00<?, ?it/s]

current doc id: 故宫
current doc id: 颐和园


Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 102 ms, sys: 8.49 ms, total: 110 ms
Wall time: 23.2 s


In [8]:
doc_summary_index.get_document_summary("颐和园")

"The provided text discusses the historical significance and architectural features of颐和园 (Summer Palace), a royal garden and palace complex located in Beijing's Haidian District, northwest of the city. It was constructed during the Qing Dynasty under Emperor Qianlong and later rebuilt during Emperor Guangxu's reign. The text highlights its status as one of the three major imperial gardens in Beijing's western suburbs alongside the other two being the圆明园 (Summer Palace) and 避暑山庄 (Mogudan Garden). \n\nThe text emphasizes that颐和园 is renowned for its skillful integration of artificial constructions with natural landscapes, which represents the pinnacle of Chinese garden artistry during its peak period. It also mentions that in 1998, the site was recognized as a UNESCO World Heritage Site.\n\nSome questions this text can answer include:\n- What is颐和园?\n- Where is颐和园 located?\n- When was颐和园 built and who were involved in its construction?\n- Why is颐和园 considered significant in Chinese histo

In [9]:
doc_summary_index.get_document_summary("故宫")

"The provided text discusses the historical significance and architectural details of the Palace Museum, also known as the Forbidden City in Beijing, China. It highlights its status as a former residence for the emperors during the Ming and Qing dynasties, emphasizing its construction history which began in 1406 under Emperor Yongle and was completed by 1420.\n\nThe text provides information about the Palace Museum's size, stating that it covers an area of approximately 720,000 square meters with a building footprint of around 150,000 square meters. It mentions that there are over 9,000 rooms and more than 70 palaces within its walls.\n\nThe text also describes the layout of the Palace Museum, noting its central location along Beijing's axis, surrounded by a 10-meter-high wall with a 59-meter-wide moat outside. It mentions that there are four main gates: the southern gate is the Gate of Heavenly Purity (Wumen), the northern gate is the Gate of Supreme Harmony (Renmen), and the eastern 

### 使用中文提示词，不会生成 QA 内容

In [11]:
%%time

from llama_index.core import PromptTemplate

qa_prompt_tmpl = (
    "上下文信息如下。\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "根据上下文信息而非先验知识，"
    "回答查询。\n"
    # "请同时以 {tone_name} 的语气写出答案。\n"
    "回答请使用中文。\n"
    "查询：{query_str}\n"
    "答案："
)

qa_prompt = PromptTemplate(qa_prompt_tmpl)

refine_prompt_tmpl=(
    "原始查询如下： {query_str}\n"
    "我们已提供现有答案： {existing_answer}\n"
    "我们有机会通过以下更多上下文来改进现有答案 "
    "（仅在需要时）。\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "鉴于新的上下文，改进原始答案以更好地"
    "回答查询。"
    "请使用中文回答。"
    "如果上下文无用，则返回原始答案。\n"
    "改进后的答案： "
)
refine_prompt=PromptTemplate(refine_prompt_tmpl)

summary_prompt_tmpl=(
    "Write a summary of the following. Try to use only the "
    "information provided. "
    "Try to include as many key details as possible.\n"
    "请使用中文回答。"
    "\n"
    "\n"
    "{context_str}\n"
    "\n"
    "\n"
    'SUMMARY:"""\n'
)
summary_prompt=PromptTemplate(summary_prompt_tmpl)


CPU times: user 105 µs, sys: 10 µs, total: 115 µs
Wall time: 118 µs


In [12]:
%%time

# default mode of building the index
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
    text_qa_template=qa_prompt,
    refine_template=refine_prompt,
    summary_template=summary_prompt,
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    data_docs,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/2 [00:00<?, ?it/s]

current doc id: 故宫
current doc id: 颐和园


Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 47.1 ms, sys: 507 µs, total: 47.6 ms
Wall time: 9.12 s


In [13]:
doc_summary_index.get_document_summary("颐和园")

'颐和园是位于中国北京市海淀区西北的一座大型皇家园林，占地297公顷，原名清漪园，建于清朝乾隆年间并曾在光绪年间进行重建。它曾属于清朝北京西郊的三山五园之一。颐和园以其人工建筑与自然山水巧妙结合的造园艺术著称，并在1998年被评为世界文化遗产。它是清朝皇家行宫，代表了中国园林艺术顶峰时期的成就。'

In [14]:
doc_summary_index.get_document_summary("故宫")

'故宫，又称紫禁城，是中国明清两代二十四位皇帝的皇宫，位于北京中轴线中心，占地面积72万平方米，建筑面积约15万平方米，拥有大小宫殿七十多座和房屋九千余间。它是世界上现存规模最大、最完整的宫殿型建筑，于1987年被选入《世界文化遗产》名录，并现为故宫博物院。\n\n故宫南北长961米，东西宽753米，四周有高10米的城墙环绕，外侧是59米宽的护城河。四面设有午门、神武门、东华门和西华门等四座城门。城墙的四个角各有一座结构复杂的角楼，民间常以“九梁十八柱七十二条脊”来形容其复杂性。\n\n故宫内的建筑分为外朝和内廷两部分。外朝中心包括太和殿、中和殿、保和殿三大殿，是国家举行大典礼的地方；三大殿两侧有文华殿、武英殿等辅助建筑。内廷的中心为乾清宫、交泰殿、坤宁宫，统称后三宫，是皇帝和皇后居住的正宫，其后有御花园。东六宫和西六宫则是后妃们的居所。\n\n此外，故宫还有外东路和外西路两部分建筑。故宫不仅是国家的重点文物保护单位，也是首批国家5A级旅游景区，并以其丰富的历史文化遗产吸引了全球游客的关注。'

## 召回

### 高级查询

In [ ]:
使用默认的嵌入实现召回。

In [22]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    similarity_top_k=5
)

CPU times: user 150 µs, sys: 0 ns, total: 150 µs
Wall time: 152 µs


In [23]:
%%time

response = query_engine.query("故宫是何时建造的？")
response.print_response_stream()

故宫，又称紫禁城，始建于明成祖永乐四年（1406年），于永乐十八年（1420年）落成。CPU times: user 60.2 ms, sys: 4.35 ms, total: 64.6 ms
Wall time: 1.17 s


In [24]:
%%time

response = query_engine.query("故宫的布局是如何划分的？")
response.print_response_stream()

故宫的布局分为外朝和内廷两个主要部分。

- **外朝**：位于中轴线上的中心位置，包括太和殿、中和殿、保和殿三大殿。这三座宫殿统称为“三大殿”，是国家举行大典礼的重要场所。三大殿两侧辅以文华殿和武英殿两组建筑，进一步支撑了外朝的功能。

- **内廷**：位于故宫的后部，其核心区域包括乾清宫、交泰殿、坤宁宫，统称为“后三宫”，是皇帝和皇后居住的主要宫殿。在后三宫之后设有御花园供休息娱乐。东六宫和西六宫则分别位于东侧和西侧，作为后妃们的居住区。

此外，故宫还有外东路和外西路两部分建筑，进一步丰富了其空间布局和功能划分。CPU times: user 248 ms, sys: 29.4 ms, total: 277 ms
Wall time: 4.18 s


In [25]:
%%time

response = query_engine.query("颐和园如何展示中国的园林设计艺术？")
response.print_response_stream()

颐和园以其人工建筑与自然山水巧妙结合的造园手法著称，充分展示了中国园林设计的艺术。它位于中国北京市海淀区西北部，占地面积297公顷，是清朝时期的皇家行宫和大型皇家园林。颐和园在乾隆年间开始修建（原名清漪园），并在光绪年间进行了重建。作为中国园林艺术顶峰时期的代表，颐和园通过精心规划的景观、建筑布局以及与自然环境的和谐融合，体现了中国传统园林设计的独特魅力。

颐和园的设计中融入了山水画意，利用地形、水系、桥梁、亭台楼阁等元素，创造出既壮观又不失细腻的空间体验。例如，昆明湖作为主要水域，通过长堤、岛屿、廊桥与周围的山峦相连接，形成了一幅生动的山水画卷。此外，颐和园内的建筑如佛香阁、长廊等，不仅具有实用功能，还融入了艺术审美，体现了中国古典园林中对空间布局、色彩搭配以及装饰细节的高度关注。

总之，颐和园通过其独特的景观设计、精巧的建筑构造以及与自然环境的完美融合，展示了中国传统园林设计的艺术精髓。CPU times: user 295 ms, sys: 39.4 ms, total: 334 ms
Wall time: 5.59 s


### 基于 LLM 的召回

In [26]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 8.11 µs


In [27]:
%%time

retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
    # choice_select_prompt=None,
    # choice_batch_size=10,
    # choice_top_k=1,
    # format_node_batch_fn=None,
    # parse_choice_select_answer_fn=None,
)

retrieved_nodes = retriever.retrieve("颐和园如何展示中国的园林设计艺术？")

CPU times: user 3.28 ms, sys: 3.68 ms, total: 6.96 ms
Wall time: 3 s


In [28]:
len(retrieved_nodes)

1

In [29]:
retrieved_nodes[0].score

10.0

In [30]:
retrieved_nodes[0].node.get_text()

'颐和园\n\n颐和园是清朝的皇家行宫和大型皇家园林，位于中国北京市海淀区西北，占地297公顷。颐和园修建于清朝乾隆年间（原名清漪园）、重建于光绪年间，曾属于清朝北京西郊三山五园之一。颐和园素以人工建筑与自然山水巧妙结合的造园手法著称于世，是中国园林艺术顶峰时期的代表。[1][2]1998年，颐和园被评为世界文化遗产。'

In [33]:
%%time

from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("颐和园如何展示中国的园林设计艺术？")
# print(response)
response.print_response_stream()

颐和园以其人工建筑与自然山水巧妙结合的造园手法展示了中国的园林设计艺术。作为清朝时期的皇家行宫和大型皇家园林，它位于中国北京市海淀区西北部，占地297公顷。颐和园在乾隆年间开始修建（原名清漪园），并在光绪年间进行了重建。它曾属于清朝北京西郊三山五园之一。

颐和园的园林设计艺术体现在其对自然景观的巧妙利用与人工建筑的和谐融合上。通过精心规划的布局、独特的建筑设计以及丰富的植被配置，颐和园展现了中国古典园林的精髓。无论是宏伟的宫殿、精致的亭台楼阁，还是蜿蜒的长廊、清澈的湖泊，都体现了古代工匠高超的艺术造诣和技术水平。

1998年，颐和园被联合国教科文组织评为世界文化遗产，这一荣誉不仅证明了其在历史上的重要地位，也进一步强调了它在中国园林设计艺术中的卓越贡献。通过颐和园，人们可以领略到中国古典园林的美学理念、建筑技艺以及与自然环境相融合的设计哲学。

总之，颐和园展示了中国的园林设计艺术，通过其独特的布局、精美的建筑和对自然景观的巧妙利用，体现了中国古代园林艺术的巅峰水平。CPU times: user 333 ms, sys: 28.8 ms, total: 361 ms
Wall time: 9.39 s


### 基于嵌入的召回

In [36]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=5,
)

retrieved_nodes = retriever.retrieve("颐和园如何展示中国的园林设计艺术？")

len(retrieved_nodes)

CPU times: user 5.49 ms, sys: 399 µs, total: 5.89 ms
Wall time: 135 ms


1

In [ ]:
%%time

# use retriever as part of a query engine
from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    # streaming=True,
)

# query
response = query_engine.query("颐和园如何展示中国的园林设计艺术？")
# print(response)
response.print_response_stream()

颐和园以其人工建筑与自然山水巧妙结合的造园手法展示了中国的园林设计艺术。作为清朝时期的皇家行宫和大型皇家园林，它位于中国北京市海淀区西北部，占地面积达297公顷。颐和园在乾隆年间开始修建（原名清漪园），并在光绪年间进行了重建。曾属于清朝北京西郊三山五园之一。

颐和园的园林设计艺术体现在以下几个方面：

1. **人工与自然融合**：颐和园的设计充分考虑了自然环境，将人工建筑与周围的山水巧妙结合，创造出既有人工雕琢之美的宫殿、亭台楼阁，又有自然野趣的湖泊、山林景观。

2. **对称布局**：中国传统园林设计中常采用对称布局，颐和园也不例外。其主要轴线从南到北贯穿整个园区，包括著名的长廊、佛香阁等标志性建筑，展现了严谨而和谐的空间布局。

3. **借景与隔景**：颐和园在设计时运用了中国古典园林中的“借景”和“隔景”手法，通过巧妙的视线引导和空间分割，使得有限的空间中能够展现出无限的景观变化，增加了游览过程中的趣味性和探索性。

4. **建筑艺术**：颐和园内的建筑风格多样，融合了汉、满等民族特色，体现了中国古典园林建筑的艺术成就。如佛香阁、昆明湖畔的十七孔桥等，都是具有代表性的建筑作品。

5. **水景设计**：作为皇家园林，水景是颐和园不可或缺的一部分。昆明湖是园区的核心水域，通过人工开凿和自然湖泊相结合的方式，形成了丰富的水面景观，与周边山林相映成趣。

综上所述，颐和园不仅是中国园林艺术的杰出代表，也是世界文化遗产之一，其独特的设计手法、和谐的空间布局以及精美的建筑艺术，充分展示了中国园林设计的艺术